# Feedback loop

In [53]:
import numpy as np
import pandas as pd
import random

# Set seed for reproducibility
random.seed(42)
np.random.seed(42)

# --- Settings ---
num_agents = 100
num_weeks = 50

# Memory decay weights for different lengths
memory_decay_weights_map = {
    2: [0.7, 0.3],
    3: [0.5, 0.3, 0.2],
    4: [0.4, 0.3, 0.2, 0.1]
}

# --- Initialize agents with random memory lengths and random risk tolerance ---
agents = [{
    'Agent_ID': i,
    'Strategy': random.choice(['Optimist', 'Pessimist', 'Average']),
    'Memory_Length': random.choice([2, 3, 4]),
    'Risk_Tolerance': random.randint(50, 75)  # random tolerance per agent
} for i in range(num_agents)]

attendance_history = []
records = []

# --- Simulation ---
for week in range(1, num_weeks + 1):
    decisions = {}
    
    for agent in agents:
        mem_len = agent['Memory_Length']
        strat = agent['Strategy']
        rt = agent['Risk_Tolerance']
        
        # extract memory slice
        memory = attendance_history[-mem_len:] if len(attendance_history) >= mem_len else []
        weights = memory_decay_weights_map[mem_len]
        
        # default decision if not enough history
        if not memory:
            if strat == 'Optimist':
                decision = 'Go'
            elif strat == 'Pessimist':
                decision = 'Stay'
            else:  # Average
                decision = 'Go' if random.random() < 0.5 else 'Stay'
            A_w = None
        else:
            # compute weighted (decayed) attendance
            A_w = sum(w * a for w, a in zip(weights, memory))
            
            if strat == 'Optimist':
                # go if any one decayed attendance <= own tolerance
                decision = 'Go' if A_w <= rt else 'Stay'
            
            elif strat == 'Pessimist':
                # check trend across memory
                trend_ok = memory[-1] <= (sum(memory[:-1]) / len(memory[:-1]))
                decision = 'Go' if (A_w <= rt and trend_ok) else 'Stay'
            
            else:  # Average Thinker
                decision = 'Go' if A_w <= rt else 'Stay'
        
        decisions[agent['Agent_ID']] = decision
        
        # record this agent-week
        records.append({
            'Week': week,
            'Agent_ID': agent['Agent_ID'],
            'Strategy': strat,
            'Memory_Length': mem_len,
            'Risk_Tolerance': rt,
            'A_weighted': A_w,
            'Decision': decision,
            'Actual_Attendance': None
        })
    
    # compute actual attendance and update history
    actual = sum(1 for d in decisions.values() if d == 'Go')
    attendance_history.append(actual)
    
    # fill actual attendance in records for this week
    for rec in records[-num_agents:]:
        rec['Actual_Attendance'] = actual

# Create DataFrame
df = pd.DataFrame(records)


In [54]:
# Save the DataFrame to CSV
csv_path = "../data/el_farol.csv"
df.to_csv(csv_path, index=False)

In [55]:
# Reload the CSV to confirm content
df_loaded = pd.read_csv(csv_path)
df_loaded.head()

,Week,Agent_ID,Strategy,Memory_Length,Risk_Tolerance,A_weighted,Decision,Actual_Attendance
0,1,0,Average,2,50,NaN,Go,53
1,1,1,Average,3,57,NaN,Go,53
2,1,2,Optimist,2,73,NaN,Go,53
3,1,3,Optimist,4,73,NaN,Go,53
4,1,4,Average,2,68,NaN,Go,53


In [67]:
# Filter the DataFrame for rows where Agent_ID == 1
agent_1_data = df[df["Agent_ID"] == 1]
agent_1_data.head(50)

,Week,Agent_ID,Strategy,Memory_Length,Risk_Tolerance,A_weighted,Decision,Actual_Attendance
1,1,1,Average,3,57,NaN,Go,53
101,2,1,Average,3,57,NaN,Go,53
201,3,1,Average,3,57,NaN,Stay,65
301,4,1,Average,3,57,55.4,Go,51
401,5,1,Average,3,57,56.2,Go,69
501,6,1,Average,3,57,61.6,Stay,52
601,7,1,Average,3,57,56.6,Go,55
701,8,1,Average,3,57,61.1,Stay,65
801,9,1,Average,3,57,55.5,Go,49
901,10,1,Average,3,57,56.8,Go,69


# Feed forward

In [2]:
import numpy as np
import pandas as pd
import random

# Set seed for reproducibility
random.seed(42)
np.random.seed(42)

# --- Settings ---
num_agents = 100
num_weeks = 50

# Memory decay weights for different lengths
memory_decay_weights_map = {
    2: [0.7, 0.3],
    3: [0.5, 0.3, 0.2],
    4: [0.4, 0.3, 0.2, 0.1]
}

# --- Initialize agents with random memory lengths and random risk tolerance ---
agents = [{
    'Agent_ID': i,
    'Strategy': random.choice(['Optimist', 'Pessimist', 'Average']),
    'Memory_Length': random.choice([2, 3, 4]),
    'Risk_Tolerance': random.randint(50, 75)  # random tolerance per agent
} for i in range(num_agents)]

attendance_history = []
records = []

# --- Simulation (feed‑forward) ---
for week in range(1, num_weeks + 1):
    # 1. Exogenous attendance
    actual = random.randint(0, num_agents)  # random attendance each week
    attendance_history.append(actual)
    
    decisions = {}
    # 2. Agents decide based on this attendance history
    for agent in agents:
        mem_len = agent['Memory_Length']
        strat = agent['Strategy']
        rt = agent['Risk_Tolerance']
        
        # extract memory slice
        memory = attendance_history[-mem_len:] if len(attendance_history) >= mem_len else []
        weights = memory_decay_weights_map[mem_len]
        
        # default decision if not enough history
        if len(memory) < mem_len:
            if strat == 'Optimist':
                decision = 'Go'
            elif strat == 'Pessimist':
                decision = 'Stay'
            else:  # Average
                decision = 'Go' if random.random() < 0.5 else 'Stay'
            A_w = None
        else:
            # compute weighted (decayed) attendance
            A_w = sum(w * a for w, a in zip(weights, memory))
            
            if strat == 'Optimist':
                decision = 'Go' if A_w <= rt else 'Stay'
            elif strat == 'Pessimist':
                trend_ok = memory[-1] <= (sum(memory[:-1]) / len(memory[:-1]))
                decision = 'Go' if (A_w <= rt and trend_ok) else 'Stay'
            else:  # Average Thinker
                decision = 'Go' if A_w <= rt else 'Stay'
        
        decisions[agent['Agent_ID']] = decision
        
        # record this agent-week
        records.append({
            'Week': week,
            'Agent_ID': agent['Agent_ID'],
            'Strategy': strat,
            'Memory_Length': mem_len,
            'Risk_Tolerance': rt,
            'A_weighted': A_w,
            'Decision': decision,
            'Actual_Attendance': actual
        })

# Create DataFrame
df = pd.DataFrame(records)

# Optionally display or save
print(df.head())
# Save the DataFrame to CSV
csv_path = "../data/ff_el_farol.csv"
df.to_csv(csv_path, index=False)

   Week  Agent_ID  Strategy  Memory_Length  Risk_Tolerance  A_weighted  \
0     1         0   Average              2              50         NaN   
1     1         1   Average              3              57         NaN   
2     1         2  Optimist              2              73         NaN   
3     1         3  Optimist              4              73         NaN   
4     1         4   Average              2              68         NaN   

  Decision  Actual_Attendance  
0     Stay                 24  
1       Go                 24  
2       Go                 24  
3       Go                 24  
4       Go                 24  
